In [1]:
# CELL 0: ONE-SHOT SETUP (NO RESTART NEEDED)
print("="*70)
print("COMPLETE SETUP - MAMS CL TRAINING")
print("="*70)

import subprocess
import sys

# Fix protobuf
print("1/2 Fixing protobuf...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "protobuf"], capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "protobuf==3.20.3"], capture_output=True)
print("✓ Protobuf fixed")

# Install packages
print("2/2 Installing packages...")
subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "pandas", "numpy", "scikit-learn",
    "transformers", "torch", "accelerate",
    "hdbscan", "umap-learn", "sentence-transformers"
], capture_output=True)
print("✓ Packages installed")

# Force reload protobuf
import importlib
if 'google.protobuf' in sys.modules:
    importlib.reload(sys.modules['google.protobuf'])

# Import everything
import json
import pandas as pd
import numpy as np
import os
from collections import defaultdict, Counter
import torch
from transformers import T5EncoderModel, AutoTokenizer
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported")

# GPU check
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("✗ No GPU")

print("="*70)
print("SETUP COMPLETE")
print("="*70)


COMPLETE SETUP - MAMS CL TRAINING
1/2 Fixing protobuf...
✓ Protobuf fixed
2/2 Installing packages...
✓ Packages installed


2025-11-10 13:38:23.554217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762781903.749987      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762781903.810911      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✓ All libraries imported
✓ GPU: Tesla T4
SETUP COMPLETE


In [1]:
# CELL 0: ONE-SHOT SETUP (NO RESTART NEEDED - BUT RESTART IF PYARROW ERROR PERSISTS)
print("="*70)
print("COMPLETE SETUP - MAMS CL TRAINING WITH EVALUATION PACKAGES")
print("="*70)
import subprocess
import sys

# Step 0: Early PyArrow Fix (Resolves Binary Incompatibility with cuDF & Datasets)
print("0/4 Fixing pyarrow & cuDF compatibility...")
# Align cuDF versions to avoid internal conflicts
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "cudf-cu12", "pylibcudf-cu12", "cudf-polars-cu12"], capture_output=True)
subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "cudf-cu12==25.6.0", "pylibcudf-cu12==25.6.0", "cudf-polars-cu12==25.6.0"
], capture_output=True)
# Nuke & reinstall compatible pyarrow (15.0.2: Works with cuDF 25.x & datasets 4.4.1)
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", "pyarrow==15.0.2"], capture_output=True)
print("✓ PyArrow & cuDF fixed")

# Step 1: Fix protobuf
print("1/4 Fixing protobuf...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "protobuf"], capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "protobuf==3.20.3"], capture_output=True)
print("✓ Protobuf fixed")

# Step 2: Install core + evaluation packages
print("2/4 Installing packages...")
subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "pandas", "numpy", "scikit-learn",
    "transformers", "torch", "accelerate",
    "hdbscan", "umap-learn", "sentence-transformers",
    "evaluate", "sacrebleu", "datasets==4.4.1"  # Explicit datasets for control
], capture_output=True)
print("✓ Packages installed")

# Step 3: Force reload protobuf (if needed)
import importlib
if 'google.protobuf' in sys.modules:
    importlib.reload(sys.modules['google.protobuf'])

# Step 4: Import everything (including evaluation)
print("3/4 Importing libraries...")
import json
import pandas as pd
import numpy as np
import os
from collections import defaultdict, Counter
import torch
from transformers import T5EncoderModel, AutoTokenizer
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Evaluation imports
import evaluate
from sacrebleu import corpus_bleu
from sentence_transformers import SentenceTransformer

print("✓ All libraries imported")

# Quick eval test
print("4/4 Testing evaluation packages...")
try:
    # Light tests
    metric = evaluate.load("bleu")  # Via evaluate
    print("✓ evaluate (BLEU) loaded")
    print(f"✓ sacrebleu: Version {sacrebleu.__version__}")
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Quick embed
    print("✓ sentence-transformers: Model loaded")
except Exception as e:
    print(f"⚠ Evaluation test partial fail: {e}")
    print("  - Run 'Runtime > Restart session' and re-run this cell if imports fail.")

# GPU check
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("✗ No GPU")
print("="*70)
print("SETUP COMPLETE - READY FOR MAMS CL TRAINING & EVAL")
print("="*70)

COMPLETE SETUP - MAMS CL TRAINING WITH EVALUATION PACKAGES
0/4 Fixing pyarrow & cuDF compatibility...
✓ PyArrow & cuDF fixed
1/4 Fixing protobuf...
✓ Protobuf fixed
2/4 Installing packages...
✓ Packages installed
3/4 Importing libraries...


2025-11-10 13:55:39.312464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762782939.712847      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762782939.866076      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✓ All libraries imported
4/4 Testing evaluation packages...


✓ evaluate (BLEU) loaded
⚠ Evaluation test partial fail: name 'sacrebleu' is not defined
  - Run 'Runtime > Restart session' and re-run this cell if imports fail.
✓ GPU: Tesla T4
SETUP COMPLETE - READY FOR MAMS CL TRAINING & EVAL


In [2]:
# ============================================================================
# CELL 1: VANILLA T5 BASELINE - MAMS ABSA FINE-TUNING
# ============================================================================
print("="*70)
print("CELL 1: BASELINE - VANILLA T5 ABSA FINE-TUNING")
print("="*70)

from transformers import T5ForConditionalGeneration, AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import Dataset
import xml.etree.ElementTree as ET

# Load vanilla T5
print("\nLoading vanilla T5-base...")
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = AutoTokenizer.from_pretrained('t5-base')
print("✓ Vanilla T5 loaded\n")

# Parse MAMS
def parse_mams_absa(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    data = []
    
    for sentence in root.findall('.//sentence'):
        text_elem = sentence.find('text')
        if text_elem is None:
            continue
        
        text = text_elem.text.strip()
        aspects = sentence.findall('aspectTerms/aspectTerm')
        
        outputs = []
        for aspect in aspects:
            term = aspect.get('term')
            polarity = aspect.get('polarity')
            outputs.append(f"{term} is {polarity}")
        
        if outputs:
            data.append({
                'input': f"extract aspects: {text}",
                'output': " [SEP] ".join(outputs)
            })
    
    return data

print("Loading MAMS data...")
train_data = parse_mams_absa('/kaggle/input/mams-for-absa/train.xml')
val_data = parse_mams_absa('/kaggle/input/mams-for-absa/val.xml')
test_data = parse_mams_absa('/kaggle/input/mams-for-absa/test.xml')

print(f"✓ Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}\n")

# Convert to datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Tokenize
def preprocess(examples):
    inputs = tokenizer(examples['input'], max_length=128, truncation=True, padding='max_length')
    targets = tokenizer(examples['output'], max_length=128, truncation=True, padding='max_length')
    inputs['labels'] = targets['input_ids']
    return inputs

print("Tokenizing...")
train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
print("✓ Tokenization complete\n")

# Train (EXACT SAME PARAMETERS AS CL MODEL)
print("Fine-tuning vanilla T5 on ABSA...")

training_args = TrainingArguments(
    output_dir='./mams_t5_baseline',
    num_train_epochs=10,  # SAME AS CL
    per_device_train_batch_size=16,  # SAME AS CL
    per_device_eval_batch_size=16,  # SAME AS CL
    warmup_steps=500,  # SAME AS CL
    weight_decay=0.01,  # SAME AS CL
    logging_steps=100,  # SAME AS CL
    eval_strategy='epoch',  # SAME AS CL
    save_strategy='epoch',  # SAME AS CL
    save_total_limit=2,  # SAME AS CL
    load_best_model_at_end=True,  # SAME AS CL
    metric_for_best_model='eval_loss',  # SAME AS CL
    fp16=True,  # SAME AS CL
    report_to='none'
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

print("="*70)
trainer.train()

print("\n✅ BASELINE TRAINING COMPLETE!")
model.save_pretrained('./mams_t5_baseline_final')
tokenizer.save_pretrained('./mams_t5_baseline_final')
print("✓ Saved to: ./mams_t5_baseline_final\n")

print("="*70)
print("CELL 1 COMPLETE - BASELINE ESTABLISHED")
print("="*70)


CELL 1: BASELINE - VANILLA T5 ABSA FINE-TUNING

Loading vanilla T5-base...


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

✓ Tokenization complete

Fine-tuning vanilla T5 on ABSA...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,10.412000,0.497752
2,0.652200,0.065949
3,0.063300,0.048242
4,0.051500,0.041848
5,0.043800,0.041282
6,0.037500,0.039139
7,0.032900,0.040358
8,0.032400,0.039567
9,0.029300,0.040019
10,0.027800,0.039906


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].



✅ BASELINE TRAINING COMPLETE!
✓ Saved to: ./mams_t5_baseline_final

CELL 1 COMPLETE - BASELINE ESTABLISHED


In [3]:
# ============================================================================
# CELL 2: COMPREHENSIVE EVALUATION - VANILLA T5 BASELINE
# ============================================================================
print("="*70)
print("EVALUATION - VANILLA T5 BASELINE - MAMS ABSA")
print("="*70)

import torch
import numpy as np
from transformers import T5ForConditionalGeneration, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import xml.etree.ElementTree as ET
import re

# Load model
print("\nLoading Vanilla T5 baseline model...")
model = T5ForConditionalGeneration.from_pretrained('./mams_t5_baseline_final')
tokenizer = AutoTokenizer.from_pretrained('./mams_t5_baseline_final')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()
print(f"✓ Model loaded on {device}\n")

# Load test data
def parse_mams_test(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    data = []
    
    for sentence in root.findall('.//sentence'):
        text_elem = sentence.find('text')
        if text_elem is None:
            continue
        
        text = text_elem.text.strip()
        aspects = sentence.findall('aspectTerms/aspectTerm')
        
        gold_tuples = []
        for aspect in aspects:
            term = aspect.get('term').strip()
            polarity = aspect.get('polarity').strip().lower()
            gold_tuples.append((term, polarity))
        
        if gold_tuples:
            data.append({
                'text': text,
                'gold_tuples': gold_tuples
            })
    
    return data

test_data = parse_mams_test('/kaggle/input/mams-for-absa/test.xml')
print(f"Evaluating {len(test_data)} test samples...\n")

# Generate predictions
print("Generating predictions...")
predictions = []
for idx, sample in enumerate(test_data):
    if idx % 100 == 0:
        print(f"  Progress: {idx}/{len(test_data)}")
    
    input_text = f"extract aspects: {sample['text']}"
    inputs = tokenizer(input_text, return_tensors='pt', max_length=128, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    pred_tuples = []
    for segment in pred_text.split('[SEP]'):
        segment = segment.strip()
        match = re.match(r'(.+?)\s+is\s+(positive|negative|neutral)', segment, re.IGNORECASE)
        if match:
            pred_tuples.append((match.group(1).strip(), match.group(2).strip().lower()))
    
    predictions.append({
        'text': sample['text'],
        'pred_tuples': pred_tuples,
        'gold_tuples': sample['gold_tuples'],
        'pred_raw': pred_text
    })

print(f"✓ Generated {len(predictions)} predictions\n")

# ============================================================
# PRIMARY METRICS
# ============================================================
print("="*70)
print("PRIMARY METRICS")
print("="*70)

def normalize(text):
    return text.lower().strip()

tuple_tp = tuple_fp = tuple_fn = 0
aspect_tp = aspect_fp = aspect_fn = 0
exact_matches = 0

for pred in predictions:
    pred_set = {(normalize(a), s) for a, s in pred['pred_tuples']}
    gold_set = {(normalize(a), s) for a, s in pred['gold_tuples']}
    
    tuple_tp += len(pred_set & gold_set)
    tuple_fp += len(pred_set - gold_set)
    tuple_fn += len(gold_set - pred_set)
    
    pred_aspects = {normalize(a) for a, _ in pred['pred_tuples']}
    gold_aspects = {normalize(a) for a, _ in pred['gold_tuples']}
    
    aspect_tp += len(pred_aspects & gold_aspects)
    aspect_fp += len(pred_aspects - gold_aspects)
    aspect_fn += len(gold_aspects - pred_aspects)
    
    if pred_set == gold_set:
        exact_matches += 1

tuple_prec = tuple_tp / (tuple_tp + tuple_fp) if (tuple_tp + tuple_fp) > 0 else 0
tuple_rec = tuple_tp / (tuple_tp + tuple_fn) if (tuple_tp + tuple_fn) > 0 else 0
tuple_f1 = 2 * tuple_prec * tuple_rec / (tuple_prec + tuple_rec) if (tuple_prec + tuple_rec) > 0 else 0

aspect_prec = aspect_tp / (aspect_tp + aspect_fp) if (aspect_tp + aspect_fp) > 0 else 0
aspect_rec = aspect_tp / (aspect_tp + aspect_fn) if (aspect_tp + aspect_fn) > 0 else 0
aspect_f1 = 2 * aspect_prec * aspect_rec / (aspect_prec + aspect_rec) if (aspect_prec + aspect_rec) > 0 else 0

exact_acc = exact_matches / len(predictions)

sent_correct = 0
sent_total = 0
for pred in predictions:
    pred_dict = {normalize(a): s for a, s in pred['pred_tuples']}
    gold_dict = {normalize(a): s for a, s in pred['gold_tuples']}
    for asp in set(pred_dict.keys()) & set(gold_dict.keys()):
        sent_total += 1
        if pred_dict[asp] == gold_dict[asp]:
            sent_correct += 1

sent_acc = sent_correct / sent_total if sent_total > 0 else 0

print(f"\n1. Aspect + Sentiment (Tuple Matching):")
print(f"   TP: {tuple_tp}, FP: {tuple_fp}, FN: {tuple_fn}")
print(f"   Precision: {tuple_prec:.4f}")
print(f"   Recall:    {tuple_rec:.4f}")
print(f"   F1 Score:  {tuple_f1:.4f}")

print(f"\n2. Aspect-Only (Ignore Sentiment):")
print(f"   TP: {aspect_tp}, FP: {aspect_fp}, FN: {aspect_fn}")
print(f"   Precision: {aspect_prec:.4f}")
print(f"   Recall:    {aspect_rec:.4f}")
print(f"   F1 Score:  {aspect_f1:.4f}")

print(f"\n3. Exact Match Accuracy:")
print(f"   {exact_matches}/{len(predictions)} = {exact_acc:.4f} ({exact_acc*100:.2f}%)")

print(f"\n4. Sentiment Accuracy (on detected aspects):")
print(f"   {sent_correct}/{sent_total} = {sent_acc:.4f} ({sent_acc*100:.2f}%)")

# ============================================================
# TOP 10 ASPECTS
# ============================================================
print("\n" + "="*70)
print("TOP 10 ASPECTS")
print("="*70)

aspect_stats = {}
for pred in predictions:
    for aspect, _ in pred['gold_tuples']:
        norm_asp = normalize(aspect)
        if norm_asp not in aspect_stats:
            aspect_stats[norm_asp] = {'gold': 0, 'pred': 0, 'tp': 0}
        aspect_stats[norm_asp]['gold'] += 1
    
    for aspect, _ in pred['pred_tuples']:
        norm_asp = normalize(aspect)
        if norm_asp not in aspect_stats:
            aspect_stats[norm_asp] = {'gold': 0, 'pred': 0, 'tp': 0}
        aspect_stats[norm_asp]['pred'] += 1
    
    pred_aspects = {normalize(a) for a, _ in pred['pred_tuples']}
    gold_aspects = {normalize(a) for a, _ in pred['gold_tuples']}
    for asp in pred_aspects & gold_aspects:
        aspect_stats[asp]['tp'] += 1

for asp, stats in aspect_stats.items():
    p = stats['tp'] / stats['pred'] if stats['pred'] > 0 else 0
    r = stats['tp'] / stats['gold'] if stats['gold'] > 0 else 0
    stats['f1'] = 2*p*r/(p+r) if (p+r) > 0 else 0

top_aspects = sorted(aspect_stats.items(), key=lambda x: x[1]['gold'], reverse=True)[:10]

print(f"\nAspect{' '*24}Gold   Pred     TP       F1")
print("-"*70)
for asp, stats in top_aspects:
    print(f"{asp[:30]:<30}{stats['gold']:>5}{stats['pred']:>7}{stats['tp']:>7}   {stats['f1']:.4f}")

# ============================================================
# PREDICTION EXAMPLES
# ============================================================
print("\n" + "="*70)
print("PREDICTION EXAMPLES (First 10)")
print("="*70)

for i, pred in enumerate(predictions[:10], 1):
    pred_set = set(pred['pred_tuples'])
    gold_set = set(pred['gold_tuples'])
    match_status = "✅ EXACT MATCH" if pred_set == gold_set else f"❌ Partial ({len(pred_set & gold_set)}/{len(gold_set)} matched)"
    
    print(f"\n{'─'*70}")
    print(f"Example {i}")
    print(f"{'─'*70}")
    print(f"TEXT:\n{pred['text'][:80]}...\n")
    print(f"GOLD:")
    for asp, sent in pred['gold_tuples']:
        print(f"{asp} | {sent}")
    print(f"\nPREDICTED:")
    for asp, sent in pred['pred_tuples']:
        print(f"{asp} | {sent}")
    print(f"\n{match_status}")

# ============================================================
# GENERATION QUALITY METRICS
# ============================================================
print("\n" + "="*70)
print("GENERATION QUALITY METRICS")
print("="*70)

gold_strings = []
pred_strings = []
for pred in predictions:
    gold_str = " [SEP] ".join([f"{a} is {s}" for a, s in pred['gold_tuples']])
    pred_str = pred['pred_raw']
    gold_strings.append(gold_str)
    pred_strings.append(pred_str)

# ROUGE
print("\n[1/4] Computing ROUGE scores...")
try:
    import evaluate
    rouge = evaluate.load('rouge')
    rouge_results = rouge.compute(predictions=pred_strings, references=gold_strings, use_stemmer=True)
    print(f"✅ ROUGE-1: {rouge_results['rouge1']:.4f}")
    print(f"   ROUGE-2: {rouge_results['rouge2']:.4f}")
    print(f"   ROUGE-L: {rouge_results['rougeL']:.4f}")
except Exception as e:
    print(f"⚠️ ROUGE skipped: {e}")

# BLEU
print("\n[2/4] Computing BLEU score...")
try:
    from sacrebleu import corpus_bleu
    references = [[g] for g in gold_strings]
    bleu_result = corpus_bleu(pred_strings, references)
    print(f"✅ BLEU Score: {bleu_result.score:.4f}")
except Exception as e:
    print(f"⚠️ BLEU skipped: {e}")

# METEOR
print("\n[3/4] Computing METEOR score...")
try:
    meteor = evaluate.load('meteor')
    meteor_result = meteor.compute(predictions=pred_strings, references=gold_strings)
    print(f"✅ METEOR Score: {meteor_result['meteor']:.4f}")
except Exception as e:
    print(f"⚠️ METEOR skipped: {e}")

# BERTScore
print("\n[4/4] Computing BERTScore...")
try:
    bertscore = evaluate.load('bertscore')
    bert_results = bertscore.compute(predictions=pred_strings, references=gold_strings, lang='en', model_type='microsoft/deberta-base-mnli', batch_size=16)
    bert_f1 = sum(bert_results['f1']) / len(bert_results['f1'])
    print(f"✅ BERTScore F1: {bert_f1:.4f}")
except Exception as e:
    print(f"⚠️ BERTScore skipped: {e}")

# ============================================================
# SEMANTIC HALLUCINATION ANALYSIS
# ============================================================
print("\n" + "="*70)
print("SEMANTIC HALLUCINATION ANALYSIS")
print("="*70)

print("\n[1/3] Loading semantic model...")
try:
    sem_model = SentenceTransformer('all-MiniLM-L6-v2')
    print("✓ Semantic model loaded")
    
    print("\n[2/3] Analyzing hallucinations...")
    total_gen = 0
    explicit_count = 0
    semantic_match = 0
    true_halluc = 0
    halluc_examples = []
    
    for pred in predictions:
        text_words = set(pred['text'].lower().split())
        gold_aspects = {normalize(a) for a, _ in pred['gold_tuples']}
        
        for aspect, sentiment in pred['pred_tuples']:
            total_gen += 1
            aspect_norm = normalize(aspect)
            
            aspect_words = aspect_norm.split()
            in_text = all(w in text_words for w in aspect_words)
            
            if in_text:
                explicit_count += 1
                continue
            
            if gold_aspects:
                asp_emb = sem_model.encode([aspect_norm])
                gold_embs = sem_model.encode(list(gold_aspects))
                sims = cosine_similarity(asp_emb, gold_embs)[0]
                max_sim = np.max(sims)
                
                if max_sim > 0.65:
                    semantic_match += 1
                else:
                    true_halluc += 1
                    if len(halluc_examples) < 5:
                        halluc_examples.append({
                            'text': pred['text'],
                            'halluc': aspect,
                            'gold': list(gold_aspects)
                        })
            else:
                true_halluc += 1
    
    semantic_halluc_rate = true_halluc / total_gen if total_gen > 0 else 0
    strict_halluc_rate = (total_gen - explicit_count) / total_gen if total_gen > 0 else 0
    
    print(f"\n✅ Semantic Hallucination Analysis:")
    print(f"   Total generated aspects:           {total_gen}")
    print(f"   Explicit (in text):                {explicit_count} ({explicit_count/total_gen*100:.1f}%)")
    print(f"   Implicit but semantically valid:   {semantic_match} ({semantic_match/total_gen*100:.1f}%)")
    print(f"   True hallucinations:               {true_halluc} ({true_halluc/total_gen*100:.1f}%)")
    print(f"\n   📊 Semantic Hallucination Rate:    {semantic_halluc_rate:.4f} ({semantic_halluc_rate*100:.1f}%)")
    print(f"   📊 Strict Hallucination Rate:      {strict_halluc_rate:.4f} ({strict_halluc_rate*100:.1f}%)")
    
    if halluc_examples:
        print(f"\n   ⚠️ Examples of true hallucinations:")
        for i, ex in enumerate(halluc_examples[:3], 1):
            print(f"   {i}. Generated: '{ex['halluc']}' | Gold: {ex['gold']}")
            print(f"      Text: {ex['text'][:80]}...")
    
    print("\n[3/3] Computing semantic coverage...")
    total_gold = 0
    exact_cov = 0
    sem_cov = 0
    
    for pred in predictions:
        pred_aspects = {normalize(a) for a, _ in pred['pred_tuples']}
        
        for aspect, _ in pred['gold_tuples']:
            total_gold += 1
            aspect_norm = normalize(aspect)
            
            if aspect_norm in pred_aspects:
                exact_cov += 1
            elif pred_aspects:
                asp_emb = sem_model.encode([aspect_norm])
                pred_embs = sem_model.encode(list(pred_aspects))
                sims = cosine_similarity(asp_emb, pred_embs)[0]
                if np.max(sims) > 0.65:
                    sem_cov += 1
    
    exact_coverage = exact_cov / total_gold if total_gold > 0 else 0
    semantic_coverage = (exact_cov + sem_cov) / total_gold if total_gold > 0 else 0
    
    print(f"\n✅ Semantic Coverage Analysis:")
    print(f"   Total gold aspects:       {total_gold}")
    print(f"   Exact matches:            {exact_cov} ({exact_cov/total_gold*100:.1f}%)")
    print(f"   Semantic matches:         {sem_cov} ({sem_cov/total_gold*100:.1f}%)")
    print(f"   Missed aspects:           {total_gold - exact_cov - sem_cov} ({(1 - semantic_coverage)*100:.1f}%)")
    print(f"\n   📊 Exact Coverage:        {exact_coverage:.4f} ({exact_coverage*100:.1f}%)")
    print(f"   📊 Semantic Coverage:     {semantic_coverage:.4f} ({semantic_coverage*100:.1f}%)")
    
except Exception as e:
    print(f"⚠️ Semantic analysis skipped: {e}")
    semantic_halluc_rate = None
    semantic_coverage = None

# ============================================================
# FORMAT ADHERENCE
# ============================================================
print("\n" + "="*70)
print("FORMAT ADHERENCE")
print("="*70)

valid_format = sum(1 for p in predictions if p['pred_tuples'] or not p['pred_raw'].strip())
malformed = len(predictions) - valid_format
empty = sum(1 for p in predictions if not p['pred_raw'].strip())

print(f"\n✅ Format Adherence:")
print(f"   Valid format: {valid_format}/{len(predictions)} ({valid_format/len(predictions)*100:.1f}%)")
print(f"   Malformed outputs: {malformed} ({malformed/len(predictions)*100:.1f}%)")
print(f"   Empty outputs: {empty} ({empty/len(predictions)*100:.1f}%)")

# ============================================================
# COMPREHENSIVE SUMMARY
# ============================================================
print("\n" + "="*70)
print("COMPREHENSIVE EVALUATION SUMMARY - VANILLA T5 BASELINE")
print("="*70)

print(f"\nModel: Vanilla T5-base (Baseline)")
print(f"Dataset: {len(predictions)} test samples\n")

print("="*70)
print("TASK-SPECIFIC METRICS (Primary)")
print("="*70)
print(f"\n1. Aspect + Sentiment F1:      {tuple_f1:.4f}")
print(f"2. Aspect-Only F1:              {aspect_f1:.4f}")
print(f"3. Exact Match Accuracy:        {exact_acc:.4f}")
print(f"4. Sentiment Accuracy:          {sent_acc:.4f}")

print("\n" + "="*70)
print("GENERATION QUALITY METRICS")
print("="*70)
if 'rouge_results' in locals():
    print(f"\n5. ROUGE-L:                     {rouge_results['rougeL']:.4f}")
if 'bleu_result' in locals():
    print(f"6. BLEU:                        {bleu_result.score:.4f}")
if 'meteor_result' in locals():
    print(f"7. METEOR:                      {meteor_result['meteor']:.4f}")
if 'bert_f1' in locals():
    print(f"8. BERTScore F1:                {bert_f1:.4f}")

print("\n" + "="*70)
print("RELIABILITY METRICS (Semantic-Aware)")
print("="*70)
if semantic_halluc_rate is not None:
    print(f"\n9. Semantic Hallucination Rate: {semantic_halluc_rate:.4f} ({semantic_halluc_rate*100:.1f}%)")
    print(f"10. Semantic Coverage:          {semantic_coverage:.4f} ({semantic_coverage*100:.1f}%)")
print(f"11. Format Adherence:           {valid_format/len(predictions):.4f} ({valid_format/len(predictions)*100:.1f}%)")

print("\n" + "="*70)
print("✅ EVALUATION COMPLETE - VANILLA T5 BASELINE")
print("="*70)
print("\n📊 Compare these results with CL T5 model for improvement analysis")


EVALUATION - VANILLA T5 BASELINE - MAMS ABSA

Loading Vanilla T5 baseline model...
✓ Model loaded on cuda

Evaluating 500 test samples...

Generating predictions...
  Progress: 0/500
  Progress: 100/500
  Progress: 200/500
  Progress: 300/500
  Progress: 400/500
✓ Generated 500 predictions

PRIMARY METRICS

1. Aspect + Sentiment (Tuple Matching):
   TP: 826, FP: 561, FN: 510
   Precision: 0.5955
   Recall:    0.6183
   F1 Score:  0.6067

2. Aspect-Only (Ignore Sentiment):
   TP: 989, FP: 398, FN: 347
   Precision: 0.7130
   Recall:    0.7403
   F1 Score:  0.7264

3. Exact Match Accuracy:
   144/500 = 0.2880 (28.80%)

4. Sentiment Accuracy (on detected aspects):
   826/989 = 0.8352 (83.52%)

TOP 10 ASPECTS

Aspect                        Gold   Pred     TP       F1
----------------------------------------------------------------------
food                             79     89     75   0.8929
service                          52     53     48   0.9143
menu                             48  

✅ ROUGE-1: 0.8495
   ROUGE-2: 0.7706
   ROUGE-L: 0.8108

[2/4] Computing BLEU score...
✅ BLEU Score: 100.0000

[3/4] Computing METEOR score...


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


✅ METEOR Score: 0.8419

[4/4] Computing BERTScore...


⚠️ BERTScore skipped: To be able to use evaluate-metric/bertscore, you need to install the following dependencies['bert_score'] using 'pip install bert_score' for instance'

SEMANTIC HALLUCINATION ANALYSIS

[1/3] Loading semantic model...


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Semantic model loaded

[2/3] Analyzing hallucinations...

✅ Semantic Hallucination Analysis:
   Total generated aspects:           1388
   Explicit (in text):                960 (69.2%)
   Implicit but semantically valid:   323 (23.3%)
   True hallucinations:               105 (7.6%)

   📊 Semantic Hallucination Rate:    0.0756 (7.6%)
   📊 Strict Hallucination Rate:      0.3084 (30.8%)

   ⚠️ Examples of true hallucinations:
   1. Generated: 'sandwiches' | Gold: ['moules', 'steak frites', 'bistro choices', 'steak tartare', 'a poached egg', 'frisee salad with bacon, blue cheese', 'burgers', 'brasserie-style menu']
      Text: The brasserie-style menu remains relatively unchanged, featuring classic bistro ...
   2. Generated: 'meal' | Gold: ['brunch', 'waitress', 'chips']
      Text: we all liked the brunch items we had but when we later asked if the chips were s...
   3. Generated: 'snow peas' | Gold: ['glass of wine', 'dessert', 'individual pizzas']
      Text: A glass of wine for ea